In [1]:
import os

os.environ["KERAS_BACKEND"] = "tensorflow"

import re
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf
import keras
from keras import layers
from keras.applications import efficientnet
from keras.layers import TextVectorization

keras.utils.set_random_seed(111)

**Download dataet**

In [2]:
!pip install kaggle

In [4]:
!mkdir ~/.kaggle
!cp /content/kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [12]:
# Download Dataset
!kaggle datasets download -d vitngquang/uit-viic-v1-0-vietnamese-image-captioning

uit-viic-v1-0-vietnamese-image-captioning.zip: Skipping, found more recently modified local copy (use --force to force download)


In [17]:
import shutil
import os

def delete_folder_and_contents(folder_path):
    try:
        # Deleting all files and subdirectories
        for item in os.listdir(folder_path):
            item_path = os.path.join(folder_path, item)
            if os.path.isfile(item_path):
                os.unlink(item_path)
            else:
                shutil.rmtree(item_path)

        # Deleting the folder itself
        shutil.rmtree(folder_path)

        print(f"{folder_path} and its contents are successfully deleted.")

    except Exception as e:
        print(f"An error occurred: {e}")

# Replace 'folder_path' with the path to the folder you want to delete
folder_path = "/content/dataset"

delete_folder_and_contents(folder_path)


/content/dataset and its contents are successfully deleted.


In [13]:
!unzip /content/uit-viic-v1-0-vietnamese-image-captioning.zip -d /content

Streaming output truncated to the last 5000 lines.
  inflating: /content/dataset/train/images/000000534373.jpg  
  inflating: /content/dataset/train/images/000000534513.jpg  
  inflating: /content/dataset/train/images/000000534694.jpg  
  inflating: /content/dataset/train/images/000000534734.jpg  
  inflating: /content/dataset/train/images/000000534876.jpg  
  inflating: /content/dataset/train/images/000000535325.jpg  
  inflating: /content/dataset/train/images/000000535467.jpg  
  inflating: /content/dataset/train/images/000000535668.jpg  
  inflating: /content/dataset/train/images/000000536419.jpg  
  inflating: /content/dataset/train/images/000000536653.jpg  
  inflating: /content/dataset/train/images/000000537069.jpg  
  inflating: /content/dataset/train/images/000000537097.jpg  
  inflating: /content/dataset/train/images/000000537631.jpg  
  inflating: /content/dataset/train/images/000000538113.jpg  
  inflating: /content/dataset/train/images/000000538454.jpg  
  inflating: /conte

**Xử lý dataset**

In [2]:
!wget -q https://github.com/jbrownlee/Datasets/releases/download/Flickr8k/Flickr8k_Dataset.zip
!wget -q https://github.com/jbrownlee/Datasets/releases/download/Flickr8k/Flickr8k_text.zip
!unzip -qq Flickr8k_Dataset.zip
!unzip -qq Flickr8k_text.zip
!rm Flickr8k_Dataset.zip Flickr8k_text.zip

In [3]:
# Path to the images
IMAGES_PATH = "Flicker8k_Dataset"

# Desired image dimensions
IMAGE_SIZE = (299, 299)

# Vocabulary size
VOCAB_SIZE = 10000

# Fixed length allowed for any sequence
SEQ_LENGTH = 25

# Dimension for the image embeddings and token embeddings
EMBED_DIM = 512

# Per-layer units in the feed-forward network
FF_DIM = 512

# Other training parameters
BATCH_SIZE = 64
EPOCHS = 30
AUTOTUNE = tf.data.AUTOTUNE

In [4]:
def load_captions_data(filename):
    """Loads captions (text) data and maps them to corresponding images.

    Args:
        filename: Path to the text file containing caption data.

    Returns:
        caption_mapping: Dictionary mapping image names and the corresponding captions
        text_data: List containing all the available captions
    """

    with open(filename) as caption_file:
        caption_data = caption_file.readlines()
        caption_mapping = {}
        text_data = []
        images_to_skip = set()

        for line in caption_data:
            line = line.rstrip("\n")
            # Image name and captions are separated using a tab
            img_name, caption = line.split("\t")

            # Each image is repeated five times for the five different captions.
            # Each image name has a suffix `#(caption_number)`
            img_name = img_name.split("#")[0]
            img_name = os.path.join(IMAGES_PATH, img_name.strip())

            # We will remove caption that are either too short to too long
            tokens = caption.strip().split()

            if len(tokens) < 5 or len(tokens) > SEQ_LENGTH:
                images_to_skip.add(img_name)
                continue

            if img_name.endswith("jpg") and img_name not in images_to_skip:
                # We will add a start and an end token to each caption
                caption = "<start> " + caption.strip() + " <end>"
                text_data.append(caption)

                if img_name in caption_mapping:
                    caption_mapping[img_name].append(caption)
                else:
                    caption_mapping[img_name] = [caption]

        for img_name in images_to_skip:
            if img_name in caption_mapping:
                del caption_mapping[img_name]

        return caption_mapping, text_data


def train_val_split(caption_data, train_size=0.8, shuffle=True):
    """Split the captioning dataset into train and validation sets.

    Args:
        caption_data (dict): Dictionary containing the mapped caption data
        train_size (float): Fraction of all the full dataset to use as training data
        shuffle (bool): Whether to shuffle the dataset before splitting

    Returns:
        Traning and validation datasets as two separated dicts
    """

    # 1. Get the list of all image names
    all_images = list(caption_data.keys())

    # 2. Shuffle if necessary
    if shuffle:
        np.random.shuffle(all_images)

    # 3. Split into training and validation sets
    train_size = int(len(caption_data) * train_size)

    training_data = {
        img_name: caption_data[img_name] for img_name in all_images[:train_size]
    }
    validation_data = {
        img_name: caption_data[img_name] for img_name in all_images[train_size:]
    }

    # 4. Return the splits
    return training_data, validation_data


# Load the dataset
captions_mapping, text_data = load_captions_data("Flickr8k.token.txt")

# Split the dataset into training and validation sets
train_data, valid_data = train_val_split(captions_mapping)
print("Number of training samples: ", len(train_data))
print("Number of validation samples: ", len(valid_data))

Number of training samples:  6114
Number of validation samples:  1529


In [6]:
text_data

['<start> A child in a pink dress is climbing up a set of stairs in an entry way . <end>',
 '<start> A girl going into a wooden building . <end>',
 '<start> A little girl climbing into a wooden playhouse . <end>',
 '<start> A little girl climbing the stairs to her playhouse . <end>',
 '<start> A little girl in a pink dress going into a wooden cabin . <end>',
 '<start> A black dog and a spotted dog are fighting <end>',
 '<start> A black dog and a tri-colored dog playing with each other on the road . <end>',
 '<start> A black dog and a white dog with brown spots are staring at each other in the street . <end>',
 '<start> Two dogs of different breeds looking at each other on the road . <end>',
 '<start> Two dogs on pavement moving toward each other . <end>',
 '<start> A little girl covered in paint sits in front of a painted rainbow with her hands in a bowl . <end>',
 '<start> A little girl is sitting in front of a large painted rainbow . <end>',
 '<start> A small girl in the grass plays 

In [5]:
captions_mapping

{'Flicker8k_Dataset/1000268201_693b08cb0e.jpg': ['<start> A child in a pink dress is climbing up a set of stairs in an entry way . <end>',
  '<start> A girl going into a wooden building . <end>',
  '<start> A little girl climbing into a wooden playhouse . <end>',
  '<start> A little girl climbing the stairs to her playhouse . <end>',
  '<start> A little girl in a pink dress going into a wooden cabin . <end>'],
 'Flicker8k_Dataset/1001773457_577c3a7d70.jpg': ['<start> A black dog and a spotted dog are fighting <end>',
  '<start> A black dog and a tri-colored dog playing with each other on the road . <end>',
  '<start> A black dog and a white dog with brown spots are staring at each other in the street . <end>',
  '<start> Two dogs of different breeds looking at each other on the road . <end>',
  '<start> Two dogs on pavement moving toward each other . <end>'],
 'Flicker8k_Dataset/1002674143_1b742ab4b8.jpg': ['<start> A little girl covered in paint sits in front of a painted rainbow with